In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv, find_dotenv
import PyPDF2
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.question_answering import load_qa_chain
from langserve import RemoteRunnable

_ = load_dotenv(find_dotenv())

chat_model = AzureChatOpenAI(
    openai_api_version="2023-09-01-preview",
    azure_endpoint=os.getenv('AZURE_API_ENDPOINT'),
    api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    azure_deployment=os.getenv('OPENAI_DEPLOYMENT_NAME'),
    model_name=os.getenv('OPENAI_MODEL_NAME'),
    model_version=os.getenv('OPENAI_API_VERSION')
)

In [ ]:
RouterChain = (
    PromptTemplate.from_template(
        """en fonction de la question de l'utilisateur , classe la question dans le topic "meteo" ,  `moovie` ou "general"
        si la question ne s'inscrit dans aucun des 2 topics
Ne répond pas en plus d'un mot

<question>
{question}
</question>

Classification:""") | llm | StrOutputParser())

In [ ]:
GeneralChain = PromptTemplate.from_template(
    """RRépond a la question suivante:
Question: {question}
"""
) | llm

In [ ]:
def route(info):
    if "meteo" in info["topic"].lower():
        return Meteochain
    elif "moovie" in info["topic"].lower():
        return moovie_api_chain
    else:
        return GeneralChain

In [ ]:
full_chain = {"topic": RouterChain, "question": lambda x: x["question"]} 
| RunnableLambda(
    route
)

In [ ]:
def ask(q):
    reponse = full_chain.invoke({"question":q})
    print(reponse)